In [ ]:
# Install necessary packages
!pip install flask flask-ngrok diffusers omegaconf transformers accelerate scipy safetensors

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Download ngrok and set up the authtoken
!wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -qq -n ngrok-stable-linux-amd64.zip
!./ngrok authtoken "your authtoken"

In [ ]:
from flask import Flask, request, jsonify, render_template
from diffusers import StableDiffusionPipeline
import torch
import gc
from flask_ngrok import run_with_ngrok
import base64
from io import BytesIO


In [ ]:
device = "cuda"
m15= "/content/drive/MyDrive/Models/SD_1.5.safetensors"
m2="/content/drive/MyDrive/Models/SD_2.safetensors"
model15 = StableDiffusionPipeline.from_single_file(m15).to(device)
model12 = StableDiffusionPipeline.from_single_file(m2).to(device)

torch.cuda.empty_cache()
gc.collect()

In [ ]:
# Create directories
!mkdir -p /content/templates
!mkdir -p /content/static

# Copy files from Google Drive to the Colab environment
!cp "/content/drive/My Drive/Models/index.html" /content/templates/index.html
!cp "/content/drive/My Drive/Models/styles.css" /content/static/styles.css
!cp "/content/drive/My Drive/Models/script.js" /content/static/script.js


In [ ]:
# Initialize Flask app
app = Flask(__name__)
app.config['THREADS_PER_PAGE'] = 2
run_with_ngrok(app)  # Start ngrok when the app is run

In [ ]:
@app.route('/generate', methods=['POST'])
def generate():
    data = request.get_json()
    prompt = data['prompt']
    model_name = data['model']

    if model_name == "1":
      with torch.no_grad():
          generated_images = model15(prompt)
          generated_image = generated_images.images[0]

    elif model_name == "2":
      with torch.no_grad():
          generated_images = model12(prompt)
          generated_image = generated_images.images[0]

    # Convert to base64
    buffered = BytesIO()
    generated_image.save(buffered, format="PNG")
    img_str = base64.b64encode(buffered.getvalue()).decode()

    # Clear VRAM
    torch.cuda.empty_cache()
    gc.collect()

    return jsonify({'image_base64': img_str})

In [ ]:
@app.route('/')
def index():
    return render_template('index.html')

In [ ]:
if __name__ == '__main__':
    app.run()